# GI2021_DZ2

Create an AlignmentFile object for “merged-tumor.bam” from Public files gallery

Take the first read from the AlignmentFile

Inspect the fields in the AlignedSegment

Inspect the flag field

Check out the flag for some reads: https://broadinstitute.github.io/picard/explain-flags.html

Calculate:

- How many unmapped reads are in the file?

- Total number of reads

- Number of reads with mapping quality 0

- Average mapping quality for all the reads

- Average mapping quality if reads with 0 mapp quality are filtered out

First, we import `pysam` library, so we could use its `AlignmentFile` function to read `.bam` file.

In [1]:
import pysam

file = pysam.AlignmentFile('merged-tumor.bam')

Here, we create `getFirst` function in order to grab first entry from the file we read. In it we call `reset` method to reset file position to beginning of file.

In [11]:
def getFirst(collection):
    for x in collection:
        collection.reset()
        return x

firstRead = getFirst(file)

With that, we can access data in the entry.

In [12]:
dictionary = firstRead.to_dict()
for key in dictionary:
    print(f"{key}: {dictionary[key]}")

name: C0HVYACXX120402:7:1207:5722:57044
flag: 1187
ref_name: 21
ref_pos: 9483249
map_quality: 27
cigar: 76M
next_ref_name: =
next_ref_pos: 9483382
length: 209
seq: TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG
qual: ==<>@?@@>@D?>@C?>8JA5?>CC>?@???BB;H-:4A?D=DB8BDE?GBJCD@B8??E<A>CDJB@B>AA@@@C
tags: ['XA:Z:GL000217.1,-110754,76M,1;', 'BD:Z:NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'MD:Z:76', 'RG:Z:1', 'BI:Z:NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'NM:i:0', 'MQ:i:27', 'AS:i:76', 'XS:i:71']


##### Calculate:

How many unmapped reads are in the file?

This information can be found in flag field, which is an int that carries information about some properties of segment by having each property tied to a particulat bit:

0. read paired
1. read mapped in proper pair
2. read unmapped
3. mate unmapped
4. read reverse strand
5. mate reverse strand
6. first in pair
7. second in pair
8. not primary alignment
9. read fails platform/vendor quality checks
10. read is PCR or optical duplicate
11. supplementary alignment

So, in order to count unmapped reads, we need to look for the ones who have a set bit on position 2. After iteration, we also have to call `reset` method again.

In [20]:
unmappedReadFlag = 0b100
unmappedReadCount = 0

for read in file:
    if read.flag & unmappedReadFlag:
        unmappedReadCount += 1

file.reset()

print(f"Number of unmapped reads: {unmappedReadCount}")

Number of unmapped reads: 17765


Total number of reads

For this we just need to go trough the file and count the iterations.

In [24]:
totalReads = 0

for read in file:
    totalReads += 1

file.reset()

print(f"Total number of reads: {totalReads}")

Total number of reads: 2921629


Number of reads with mapping quality 0

Information about mapping quality can be found in `mapq` member

In [22]:
mapQ0Reads = 0

for read in file:
    if read.mapq == 0:
        mapQ0Reads += 1

file.reset()

print(f"Reads with mapping quality 0: {mapQ0Reads}")

Reads with mapping quality 0: 126628


Average mapping quality for all the reads

We get this by summing mapping qualities of all reads and dividing them with total number of reads which we allready have (`totalReads`).

In [25]:
totalMapQ = 0

for read in file:
    totalMapQ += read.mapq

file.reset()

print(f"Average mapping quality: {totalMapQ / totalReads}")

Average mapping quality: 55.91379158681681


Average mapping quality if reads with 0 map quality are filtered out

For this, we divide sum of mapping qualities (`totalMapQ`) with difference between number of all reads (`totalReads`) and number of reads with mapping quality 0 (`mapQ0Reads`).

In [26]:
print(f"Average mapping quality without reads with 0 map quality: {totalMapQ / (totalReads - mapQ0Reads)}")

Average mapping quality without reads with 0 map quality: 58.446975510921106
